In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
carpeta_path = r'C:\Users\William\Desktop\el algoritmo es correcto'
archivo_train = r'train_data.csv'
path_train = os.path.join(carpeta_path, archivo_train)

In [ ]:
train = pd.read_csv(
    path_train, 
    dtype={
        "tiempodeconstruido": str,
        "vista": str,
        "tipoinmueble": str,
        "tiponegocio": str,
        'valorventa': 'int64',
        # 'area': 'int64'
    }
)
train['data'] = 'train'

In [ ]:
train.shape

In [ ]:
carpeta_path = r'C:\Users\William\Desktop\el algoritmo es correcto'
archivo_test = r'test_data.csv'
path_test = os.path.join(carpeta_path, archivo_test)

In [ ]:
test = pd.read_csv(
    path_test, 
    dtype={
        "tiempodeconstruido": str,
        "vista": str,
        "tipoinmueble": str,
        "tiponegocio": str,
        'valorventa': 'int64',
        # 'area': 'int64'
    }
)
test['data'] = 'test'

In [ ]:
test.shape

In [ ]:
prueba = pd.concat([train, test], ignore_index=True)

In [ ]:
prueba = prueba[
    (prueba['porteria'].isna()) &
    (prueba['conjuntocerrado']==0) &
    (prueba['tipoinmueble']=='Casa') &
    (prueba['ascensor'].isna()) &
    ((prueba['gimnasio'].isna()) | (prueba['gimnasio']==0)) &
    (prueba['piscina'].isna()) &
    ((prueba['area']<1500) & (prueba['area']>35)) &
    ((prueba['piso']<5) | (prueba['piso'].isna())) &
    ((prueba['habitaciones']<15) & (prueba['habitaciones']>0))
]

In [ ]:
columnas_no_usar = ['ascensor', 'tipoinmueble', 'porteria', 'conjuntocerrado', 'gimnasio', 'piscina', 'valoradministracion','tiponegocio']
prueba = prueba.drop(columns = columnas_no_usar)

In [ ]:
prueba['banos'] = np.where(
    prueba['banos']==0,
    1,
    np.where(
        prueba['banos']>5,
        5,
        np.where(
            prueba['banos'].isna(),
            1,
            prueba['banos']
        )
    )
)

In [ ]:
prueba['garajes'] = np.where(
    prueba['garajes']>4,
    5,
    np.where(
        prueba['garajes']<=4,
        prueba['garajes'],
        'No define'
    )
).astype(str)

In [ ]:
prueba['piso'] = np.where(
    prueba['piso']>4,
    4,
    np.where(
        prueba['piso']<=4,
        prueba['piso'],
        'No define'
    )
).astype(str)

In [ ]:
prueba['vista'] = np.where(
    prueba['vista'].notna(),
    prueba['vista'],
    'No defina'
)

In [ ]:
prueba['zonalavanderia'] = np.where(
    prueba['deposito'].isna(),
    'No define',
    1
).astype(str)

In [ ]:
prueba['vigilancia'] = np.where(
    prueba['vigilancia'].isna(),
    'No define',
    prueba['vigilancia']
).astype(str)

In [ ]:
prueba['deposito'] = np.where(
    prueba['deposito']==0,
    'No',
    np.where(
        prueba['deposito']==1,
        'Si',
        'No define'
    )
)

In [ ]:
prueba['tiempodeconstruido'] = np.where(
    prueba['tiempodeconstruido'].isna(),
    'No define',
    prueba['tiempodeconstruido']
)

In [ ]:
columnas_valor0 = [
    'balcon', 'banoservicio', 'cuartoservicio', 'estudio', 'halldealcobas', 
    'remodelado', 'parqueaderovisitantes', 'saloncomunal','terraza']
prueba[columnas_valor0] = prueba[columnas_valor0].fillna(0)

In [ ]:
prueba.info()

In [ ]:
prueba[['habitaciones', 'banos', 'estrato']].plot(kind = 'box')

In [ ]:
prueba['y'] = prueba['valorventa']/prueba['area']

In [ ]:
prueba.groupby('data', as_index=False).id.count()

In [ ]:
prueba = prueba[prueba['y']<prueba['y'].quantile(0.95).astype('int64')]

In [ ]:
prueba[["y","estrato", "habitaciones"]].boxplot(by=["estrato", "habitaciones"], figsize=(10,10))

In [ ]:
import geopandas

In [ ]:
localidades_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\localidades.json'
localidades = geopandas.read_file(localidades_path, crs={'init':'epsg:4686'})

In [ ]:
localidades = localidades[['LocNombre', 'geometry']]

In [ ]:
estratificacion_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\manzana_estratificacion.json'
estratificacion = geopandas.read_file(estratificacion_path, crs={'init':'epsg:4686'})

In [ ]:
estratificacion = estratificacion[['ESTRATO', 'geometry']]

In [ ]:
casas = geopandas.GeoDataFrame(
    prueba, geometry=geopandas.points_from_xy(prueba.longitud, prueba.latitud), crs={'init': 'epsg:4326'})

In [ ]:
avaluo_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\avaluo'
avaluo = geopandas.read_file(avaluo_path, crs={'init':'epsg:4686'})

In [ ]:
avaluo = avaluo[['CP_TERR_AR', 'GRUPOP_TER', 'AVALUO_COM', 'AVALUO_CAT', 'geometry']]

In [ ]:
avaluo = avaluo[
    (avaluo['AVALUO_CAT']<avaluo['AVALUO_CAT'].quantile(0.99)) &
    (avaluo['AVALUO_CAT']>avaluo['AVALUO_CAT'].quantile(0.01)) &
    (avaluo['AVALUO_COM']<avaluo['AVALUO_COM'].quantile(0.99)) &
    (avaluo['AVALUO_COM']>avaluo['AVALUO_COM'].quantile(0.01))
]

In [ ]:
pd.concat(
    [avaluo[['AVALUO_CAT', 'AVALUO_COM']].quantile(np.linspace(0,1,21)).astype('int64'),
    casas[['y']].quantile(np.linspace(0,1,21)).astype('int64')],
    axis=1)

In [ ]:
interseccion1 = geopandas.sjoin(left_df=casas, right_df=estratificacion, how='left', op='intersects').drop(columns=['index_right'])

In [ ]:
interseccion1.columns

In [ ]:
interseccion2 = geopandas.sjoin(left_df=interseccion1, right_df=avaluo, how='left', op='intersects').drop(columns=['index_right'])

In [ ]:
interseccion2.columns

In [ ]:
columnas_rellenar = ['CP_TERR_AR', 'GRUPOP_TER']
interseccion2[columnas_rellenar] = interseccion2[columnas_rellenar].fillna('No define')

In [ ]:
interseccion2.info()

In [ ]:
interseccion3 = geopandas.sjoin(left_df=interseccion2, right_df=localidades, how='left', op='intersects').drop(columns=['index_right'])

In [ ]:
valorref_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\valor_ref'
valorref = geopandas.read_file(valorref_path, crs={'init':'epsg:4686'})

In [ ]:
valorref = valorref[['V_REF', 'ANO', 'geometry']]

In [ ]:
interseccion4 = geopandas.sjoin(left_df=interseccion3, right_df=valorref, how='left', op='intersects').drop(columns=['index_right'])

In [ ]:
interseccion4 = interseccion4.sort_values(['id', 'ANO']).drop_duplicates('id', keep='last')

In [ ]:
import seaborn as sns
# sns.pairplot(interseccion4[['y','CP_TERR_AR', 'GRUPOP_TER','AVALUO_COM', 'AVALUO_CAT', 'V_REF']], hue="GRUPOP_TER")

In [ ]:
# sns.pairplot(interseccion4[['y','CP_TERR_AR', 'GRUPOP_TER','AVALUO_COM', 'AVALUO_CAT', 'V_REF']], hue="CP_TERR_AR", kind="kde")

In [ ]:
# parque_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\parque'
# parque = geopandas.read_file(parque_path)

In [ ]:
# canchas_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\canchas'
# canchas = geopandas.read_file(canchas_path, crs={'init':'epsg:4686'})

In [ ]:
troncales_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\transmilenio'
troncales = geopandas.read_file(troncales_path, crs={'init':'epsg:4686'})

In [ ]:
colegios_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\colegios.csv'
colegios = pd.read_csv(colegios_path, encoding='latin1', sep=';')
colegios.X = colegios.X.str.replace(',','.').astype(float)

In [ ]:
colegios = geopandas.GeoDataFrame(
    colegios, geometry=geopandas.points_from_xy(colegios.X, colegios.Y), crs={'init': 'epsg:4326'})

In [ ]:
colegios['geometry'] = colegios.buffer(0.01)

In [ ]:
columna_colegios = ['COLEGIO ELISA BORRERO DE PASTRANA', 'CENTRO LESTONNAC - COMPAÑIA DE MARIA']
colegios_sub = colegios[colegios.NOMBRE_ESTABLECIMIENTO_EDUCATIVO.isin(columna_colegios)]

In [ ]:
bases1 = localidades.plot(color='white', edgecolor='black', figsize=(10, 10), zorder=1)
bases2 = troncales.plot(ax=bases1, figsize=(10, 10), zorder=2, color='blue', markersize=.1)
bases3 = colegios_sub.head().buffer(0.001).plot(ax=bases2, figsize=(10, 10), zorder=3, color='red',alpha=0.5)
bases4 = colegios_sub.head().plot(ax=bases3, figsize=(10, 10), zorder=3, color='red',markersize=.1)
bases5 = interseccion3[interseccion3['LocNombre']=='ENGATIVA'].sample(50, random_state=42).plot(ax=bases3, figsize=(10, 10), zorder=3, color='orange',markersize=.1)

In [ ]:
cant_colegios = geopandas.sjoin(
    left_df=casas[['id', 'geometry']], 
    right_df=colegios[['NOMBRE_ESTABLECIMIENTO_EDUCATIVO', 'geometry']], 
    how='left', 
    op='intersects').drop(columns=['index_right']).\
        groupby('id', as_index=False, dropna=False).\
        agg(**{'cantidad de colegios cerca': pd.NamedAgg('NOMBRE_ESTABLECIMIENTO_EDUCATIVO', 'count')})
cant_colegios

In [ ]:
interseccion5 = pd.merge(interseccion4, cant_colegios, how='inner', on='id')

In [ ]:
interseccion5['Cant Col. Cerca'] = np.where(
    interseccion5['cantidad de colegios cerca']<3, 
    interseccion5['cantidad de colegios cerca'].astype(str), 
    '3+')

In [ ]:
import seaborn as sns
# sns.pairplot()
g = sns.PairGrid(interseccion5[['y', 'cantidad de colegios cerca', 'Cant Col. Cerca']], hue='Cant Col. Cerca', diag_sharey=False)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot)

In [ ]:
 interseccionn = interseccion5

In [ ]:
interseccionn = interseccionn[interseccionn['LocNombre'].notna()]

In [ ]:
interseccionn.info()

In [ ]:
interseccionn['ESTRATO'] = np.where(
    interseccionn['ESTRATO'].isna() | interseccionn['ESTRATO']==0,
    np.NaN,
    interseccionn['ESTRATO']
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt

In [ ]:
train = interseccionn.loc[interseccionn['ESTRATO'].notna(),['ESTRATO', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['ESTRATO']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = KNeighborsClassifier(n_neighbors=3, metric = 'euclidean')
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

In [ ]:
valores = interseccionn.loc[interseccionn['ESTRATO'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['ESTRATO'].isnull(),'ESTRATO'] = neigh.predict(valores)

In [ ]:
train = interseccionn.loc[interseccionn['AVALUO_COM'].notna(),['AVALUO_COM', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['AVALUO_COM']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = KNeighborsRegressor(algorithm='brute', n_neighbors=5,  leaf_size=1, p=2, weights='distance', metric='chebyshev', n_jobs=-1)
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
valores = interseccionn.loc[interseccionn['AVALUO_COM'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['AVALUO_COM'].isnull(),'AVALUO_COM'] = neigh.predict(valores)

In [ ]:
train = interseccionn.loc[interseccionn['AVALUO_CAT'].notna(),['AVALUO_CAT', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['AVALUO_CAT']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
neigh = KNeighborsRegressor(algorithm='brute', n_neighbors=5,  leaf_size=1, p=2, weights='distance', metric='chebyshev', n_jobs=-1)
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
valores = interseccionn.loc[interseccionn['AVALUO_CAT'].isnull(),['latitud', 'longitud']].values
interseccionn.loc[interseccionn['AVALUO_CAT'].isnull(),'AVALUO_CAT'] = neigh.predict(valores)

In [ ]:
interseccionn.info()

In [ ]:
interseccionn.groupby(['ESTRATO'], as_index=False, dropna=False).agg(**{'Cantidad': pd.NamedAgg('latitud','count')})

In [ ]:
a = interseccionn['estrato'] - interseccionn['ESTRATO']
pd.DataFrame(a, columns=['Dif Estrato']).reset_index(drop=False).rename(columns={'index':'Cantidad'}).groupby('Dif Estrato', as_index=False, dropna=False).count()

In [ ]:
interseccionn = interseccionn.set_index('id')

In [ ]:
interseccionn.columns

In [ ]:
eliminar_columnas = ['estrato', 'geometry', 'ANO']
interseccionn = interseccionn.drop(columns=eliminar_columnas)

In [ ]:
interseccionn.loc[interseccionn["habitaciones"]<5,["y","ESTRATO", "habitaciones"]].boxplot(by=["ESTRATO", "habitaciones"], figsize=(10,10))

In [ ]:
columnas_dummy = [
    'deposito', 'garajes', 'piso', 'tiempodeconstruido', 
    'vigilancia',  'vista', 'zonalavanderia', 'LocNombre'
     ]
# 'LocNombre', 'V_REF', ,
data1 = pd.get_dummies(
    interseccionn.drop(columns=['latitud', 'longitud',   'Cant Col. Cerca', 'V_REF', 'AVALUO_COM','AVALUO_CAT','cantidad de colegios cerca', 'GRUPOP_TER', 'CP_TERR_AR']),
    drop_first=True, columns=columnas_dummy)

In [ ]:
data1.columns

In [ ]:
data_test = data1[data1['data']=='test'].drop(columns=['data'])
data1 = data1[data1['data']=='train'].drop(columns=['data'])

In [ ]:
X = data1.drop(columns=['y']).values
y = data1[['y']].values

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(data_test.shape)

In [ ]:
data1.info()

In [ ]:
mod = DecisionTreeRegressor(
    criterion='friedman_mse',  
    splitter='best', 
    max_depth=70, #None
    min_samples_split=2, 
    min_weight_fraction_leaf=0.0, 
    max_leaf_nodes=None,
    max_features='auto', #None
    min_impurity_decrease=0,
    random_state=0
).fit(X_train, y_train)

In [ ]:
mean_absolute_percentage_error(y_train, mod.predict(X_train))

In [ ]:
mean_absolute_percentage_error(y_test, mod.predict(X_test))*1000

In [ ]:
mean_absolute_percentage_error(
    data_test[['y']].values,
    mod.predict(data_test.drop(columns=['y']).values))*1000

In [ ]:
# resultados = mod.predict(data_test.drop(columns=['y']).values)

In [ ]:
# resultados_df = pd.DataFrame(resultados, index=data_test.index, columns=['valormt2_predicted'])\
#     .reset_index(drop=False).rename(columns={'index':'id'})

In [ ]:
# resultados_df.to_csv(r'C:\Users\William\Desktop\el algoritmo es correcto\resultado3.csv', index=False)

## Modelo 2

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# parameters = {
#     'n_estimators': [10,50,100,150,200,250,500],
#     'criterion': ['mse','mae'],
#     'random_state': [0]
# }
# cvrf = GridSearchCV(RandomForestRegressor(), parameters, cv=2, n_jobs=-1)

In [ ]:
# cvrf.fit(X_train, y_train)

In [ ]:
RandomForestRegressor().get_params().keys()

In [ ]:
# cvrf.best_params_

In [ ]:
from sklearn.ensemble import RandomForestRegressor
mod2 = RandomForestRegressor(random_state=0, n_jobs=-1).fit(X_train, y_train.ravel())

In [ ]:
print(mean_absolute_percentage_error(y_train, mod2.predict(X_train)))
print(mean_absolute_percentage_error(y_test, mod2.predict(X_test))*1000)
print(mean_absolute_percentage_error(
    data_test[['y']].values,
    mod2.predict(data_test.drop(columns=['y']).values))*1000)

In [ ]:
resultados = mod2.predict(data_test.drop(columns=['y']).values)*0.94
resultados_df = pd.DataFrame(resultados, index=data_test.index, columns=['valormt2_predicted'])\
    .reset_index(drop=False).rename(columns={'index':'id'})
resultados_df.to_csv(r'C:\Users\William\Desktop\el algoritmo es correcto\resultado6.csv', index=False)